# NLP Project - Web Scraping and Text Analysis of Game Reviews on Rock Paper Shotgun
## Part II. Text Analysis & Topic Modelling
### Step 2 - Topic Modelling - LSA

LSA Topic Modelling Code taken from NLP # Week 4: Topic modelling nursery rhymes with TF-IDF features and Latent Semantic Analysis (LSA)

Topic modelling with [**TF-IDF**]

In [1]:
import os
import nltk
import pandas as pd
import string
import re

from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/irenex/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/irenex/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/irenex/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
# Function originally from: https://www.programcreek.com/python/?CodeExample=get%20wordnet%20pos
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [4]:
# original code obtained from lecture notebook - Week 3 - building a stemmer and set of stop words
def customised_stemmer(word):
    stemming_rules = [
        # remove all digits
        (r"\d", "")
    ]
    for pattern, replacement in stemming_rules:
        word = re.sub(pattern, replacement, word)
    return word

In [5]:
# original code obtained from lecture notebook - Week 3 - building a stemmer and set of stop words
def load_text_documents(folder_path):
    document_texts = []
    document_labels = []
    
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".txt"):
                with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
                    text = f.read()
                
                # Apply stemming to each word
                stemmed_text = " ".join([customised_stemmer(word) for word in text.split()])

                lemmatizer = WordNetLemmatizer()
                # Apply lemmatizer to each word
                lemmitized_text = " ".join([lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in stemmed_text.split()])
                
                document_texts.append(lemmitized_text)

                document_labels.append(os.path.basename(file[:-4]))

    return document_texts, document_labels

In [6]:
folder_path = "game_reviews_rps_in_recent_years"
document_texts, document_labels = load_text_documents(folder_path)
print(f'loaded {len(document_labels)} documents')

loaded 540 documents


In [7]:
print(f'The first document is {document_labels[0]}, which goes:')
print(document_texts[0])

The first document is Quantum_League_review_(early_access), which goes:
Quantum League be an excitingly original multiplayer first-person shooter in which time repeat every seconds, allow you to collaborate with past selves. Each fifteen second of a match of competitive FPS Quantum League repeats, create it own timeline of player action which persist until the end of the third run, or until that timeline be interrupt by the action of a player in a subsequent run. It’s Clock Blockers, basically. Sometimes, you pull off supremely tactical synchronize play with version of your past self. Sometimes, you have to spend your third run kill your second run to stop yourself accidentally blowing up your first run with a grenade launcher. I like it a lot, even if I’m not sure I can adequately explain it. Once more for the people in the back, one of which happens to be me from a future timeline where I still haven’t work out the best way to sum thing up: if the Grandfather Paradox come down to “If

In [9]:
# this section has been updated after the submission of the essay, adding more stop words
english_stop_words = stopwords.words('english')
game_review_stop_words = ['cookies', 'go', 'able', 'make', 'one', 'get', 'like', 'try', 'day', 'thing', 'feel', 'look', 'yes', 'yet', 'would', 'bit', 'also', 'even', 'take', 'around', 'much', 'still', 'every', 'give', 'play', 'write', 'use', 'new', 'way', 'little', 'game', 'felt', 'always', 'story', 'behind']
stop_words = english_stop_words + game_review_stop_words

In [10]:
# ngram_range=(1,1) means considering unigrams (single words) as the basic units for analysis
vectorizer = TfidfVectorizer(stop_words=stop_words, ngram_range=(1,1))

tf_idf = vectorizer.fit_transform(document_texts)
vocab = vectorizer.get_feature_names_out()
print(f'Our bag of words is a matrix of the shape and size {tf_idf.shape}')

Our bag of words is a matrix of the shape and size (540, 27168)


In [11]:
tfidf_df = pd.DataFrame(tf_idf.toarray(), columns=vocab, index=document_labels)
tfidf_df

,_____,aa,aaa,aaaa,aaarghhhh,aahhhs,ab,abacus,abandon,abandoned,...,zoomed,zootopia,zorin,zuke,zurk,zzz,zürich,ólafsson,ólafur,δv
Quantum_League_review_(early_access),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Beacon_Pines_review:_a_cosy_horror_adventure_game_that_succeeds_on_the_strength_of_its_characters,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Season:_A_Letter_To_The_Future_review:_a_melancholy_travelogue_of_a_gorgeous_post-war_world,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Gibbon:_Beyond_The_Trees_review:_a_short,_but_heartfelt_ecological_adventure",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Payday_3_review:_this_co-op_crime_'em_up_has_lovely_levels,_but_it_won't_steal_your_heart",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Disintegration_review,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Carrion_review,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Sea_Of_Stars_review:_a_slick_RPG_that_harks_back_to_the_Chrono_Trigger_classics,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Nobody_Saves_The_World_review:_a_charming_action_RPG_plagued_by_repetition,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# iloc[0] retrieves the row at the 0th position - the first row

single_row_df = tfidf_df.iloc[0]
single_row_df = single_row_df.replace(0.0,None)#remove words with zero counts
single_row_df = single_row_df.dropna() #remove missing or NaN (Not-a-Number) values from a DataFrame or Series.
single_row_df

accidentally     0.03136
accompany       0.030969
accuracy        0.041241
action          0.052133
actually        0.015516
                  ...   
wins            0.048096
wipe            0.035312
word            0.024269
work            0.025822
world           0.012708
Name: Quantum_League_review_(early_access), Length: 348, dtype: object

In [13]:
tfidf_df = tfidf_df - tfidf_df.mean()

In [14]:
num_topics = 120
pd.options.display.max_columns=num_topics #Make sure we display them all
labels = ['topic{}'.format(i) for i in range(num_topics)] 

In [15]:
svd = TruncatedSVD(n_components = num_topics, n_iter = 100) #You can change n_iter: Higher numbers will take longer but may (or may not) give you better results
svd_topic_vectors = svd.fit_transform(tfidf_df.values)

##### Topics vs. Terms

In [16]:
topic_weights = pd.DataFrame(svd.components_.T, index=vocab, columns=labels)
topic_weights.sample(20)

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,topic11,topic12,topic13,topic14,topic15,topic16,topic17,topic18,topic19,topic20,topic21,topic22,topic23,topic24,topic25,topic26,topic27,topic28,topic29,topic30,topic31,topic32,topic33,topic34,topic35,topic36,topic37,topic38,topic39,topic40,topic41,topic42,topic43,topic44,topic45,topic46,topic47,topic48,topic49,topic50,topic51,topic52,topic53,topic54,topic55,topic56,topic57,topic58,topic59,topic60,topic61,topic62,topic63,topic64,topic65,topic66,topic67,topic68,topic69,topic70,topic71,topic72,topic73,topic74,topic75,topic76,topic77,topic78,topic79,topic80,topic81,topic82,topic83,topic84,topic85,topic86,topic87,topic88,topic89,topic90,topic91,topic92,topic93,topic94,topic95,topic96,topic97,topic98,topic99,topic100,topic101,topic102,topic103,topic104,topic105,topic106,topic107,topic108,topic109,topic110,topic111,topic112,topic113,topic114,topic115,topic116,topic117,topic118,topic119
wd,0.002029,-0.002272,0.000783,-0.002038,0.001323,0.001275,-0.000565,0.002007,0.003967,0.000837,-0.001919,-0.000963,-0.003741,0.001742,-0.000018,-0.001643,0.001400,-0.001582,0.002174,0.000798,-0.000727,-0.001650,0.002198,-0.001607,0.001434,0.001987,-0.000484,-0.002317,-0.001843,0.002195,-0.000358,0.000653,-0.000636,0.001616,-0.000366,-0.000535,0.000751,-0.001003,-0.001512,0.000793,0.001787,-0.002285,-0.000005,0.000589,-0.000279,-0.001231,-0.001042,0.000195,0.000114,0.001089,-0.000400,-0.002016,0.003399,-0.001530,-0.005341,0.000204,-0.000388,0.002873,0.001168,-0.000955,0.000148,0.000474,0.001222,-0.002115,-0.002306,0.001006,0.001196,-0.001030,-0.003215,0.001280,-0.003067,0.002302,0.000094,-0.002424,-0.001755,-0.003520,-0.005222,0.002910,0.000056,-0.001795,0.004323,-0.001975,0.001126,0.001367,-0.000567,-0.000111,-0.000552,-0.001724,-0.001437,0.002263,-0.005041,0.000888,-0.000420,-0.001217,0.000945,0.000887,-0.001800,0.001582,-0.000066,-0.000749,0.001232,-0.004729,0.001286,-0.002710,-0.001755,0.006191,0.001830,-0.000196,0.000339,0.002688,-0.000640,-0.000734,-0.005172,0.004544,-0.004028,0.004498,-0.000124,-0.005793,0.003633,-0.000752
ska,0.001887,-0.001305,-0.010397,0.003275,-0.009235,0.005012,-0.003335,-0.003340,0.002987,0.011555,0.000080,-0.004099,-0.002812,-0.000392,0.005793,-0.007080,0.004227,0.007344,-0.000583,-0.005226,0.005218,-0.003668,0.003242,-0.006260,-0.002486,0.000255,0.003958,0.000495,-0.003259,0.000648,-0.001294,-0.005225,-0.006569,-0.004937,0.007303,0.002597,-0.001812,0.004455,-0.002950,-0.004825,0.000210,0.003202,0.004964,-0.004543,-0.009525,-0.015347,-0.001180,0.004059,-0.001797,-0.002571,-0.000250,-0.003303,-0.005847,-0.004738,0.001785,0.001301,0.002940,-0.010616,-0.001521,-0.000978,0.006814,-0.003609,-0.007726,-0.012872,0.003641,-0.011445,0.000160,0.008080,-0.003939,0.003618,0.006313,-0.011383,-0.002040,0.011526,0.002961,-0.012054,-0.011260,0.003233,-0.000871,-0.003694,0.001549,0.004863,-0.004453,-0.014141,0.003242,0.005573,-0.004079,-0.001077,0.001339,0.010459,0.009185,0.005455,0.006770,0.010937,0.013992,-0.015681,0.002088,-0.003796,0.014906,-0.011837,-0.003298,0.005516,0.007669,-0.001579,0.019759,-0.010414,-0.015169,0.003872,-0.009177,-0.010375,0.001696,-0.003867,-0.002925,-0.008137,0.003108,-0.004478,-0.003071,0.001357,0.008282,-0.005907
amorous,0.001844,-0.000772,-0.004469,-0.002329,-0.005245,-0.000123,0.003565,0.000504,-0.004757,-0.003510,0.001079,0.001374,0.000626,0.006109,-0.004285,-0.003687,-0.000004,0.001346,-0.001820,-0.000843,-0.002414,-0.002206,-0.005737,-0.000913,0.001041,-0.001046,0.000715,0.001600,-0.001265,0.007105,-0.005306,-0.002840,-0.007866,0.001761,0.003604,-0.003451,-0.002985,0.000258,0.002681,-0.000843,-0.000896,0.005352,0.003733,0.003461,-0.003324,-0.000137,-0.000131,-0.006016,0.003495,-0.005020,-0.000721,-0.002054,-0.001402,0.004334,-0.000914,-0.004202,-0.003646,-0.006249,-0.004153,0.001879,-0.001833,-0.002052,-0.003191,0.004229,-0.008803,-0.002468,-0.002898,0.000648,-0.004380,-0.001146,0.002088,-0.004655,-0.003142,-0.005403,0.001282,

In [17]:
# this section has been updated after the submission of the essay
# removing more stop words didn't really improve the performance here

num_terms = 10
for i in range(num_topics):
    print("[ topic " + str(i) + " ]")
    topicName = "topic" + str(i)
    weightedlist = topic_weights.get(topicName).sort_values()[-num_terms:]
    print(weightedlist.index.values)
    print("")

[ topic 0 ]
['campaign' 'fight' 'cards' 'strategy' 'attack' 'deck' 'enemy' 'battle'
 'unit' 'card']

[ topic 1 ]
['foretales' 'detective' 'wildfrost' 'ball' 'shark' 'golf' 'cards' 'deck'
 'puzzle' 'card']

[ topic 2 ]
['station' 'colony' 'planet' 'build' 'unit' 'resource' 'crew' 'space'
 'building' 'ship']

[ topic 3 ]
['shooter' 'mode' 'weapon' 'crew' 'gun' 'space' 'card' 'ball' 'ship'
 'golf']

[ topic 4 ]
['multiplayer' 'warzone' 'cod' 'duty' 'golf' 'war' 'warfare' 'mode'
 'campaign' 'unit']

[ topic 5 ]
['level' 'club' 'city' 'unit' 'dorfromantik' 'tile' 'puzzle' 'building'
 'ball' 'golf']

[ topic 6 ]
['character' 'car' 'golfing' 'kiryu' 'persona' 'club' 'dungeon' 'yakuza'
 'ball' 'golf']

[ topic 7 ]
['club' 'detective' 'island' 'battle' 'crew' 'ship' 'ball' 'unit' 'puzzle'
 'golf']

[ topic 8 ]
['akame' 'onomichi' 'gaiden' 'persona' 'dragon' 'ishin' 'ichiban' 'unit'
 'kiryu' 'yakuza']

[ topic 9 ]
['hunt' 'map' 'hearts' 'monsties' 'sunbreak' 'rise' 'frog' 'hunter'
 'island' 'mon

##### Topics vs. Documents

In [20]:
svd_topic_vectors_df = pd.DataFrame(svd_topic_vectors, index=document_labels, columns=labels)
svd_topic_vectors_df.sample(10)

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,topic11,topic12,topic13,topic14,topic15,topic16,topic17,topic18,topic19,topic20,topic21,topic22,topic23,topic24,topic25,topic26,topic27,topic28,topic29,topic30,topic31,topic32,topic33,topic34,topic35,topic36,topic37,topic38,topic39,topic40,topic41,topic42,topic43,topic44,topic45,topic46,topic47,topic48,topic49,topic50,topic51,topic52,topic53,topic54,topic55,topic56,topic57,topic58,topic59,topic60,topic61,topic62,topic63,topic64,topic65,topic66,topic67,topic68,topic69,topic70,topic71,topic72,topic73,topic74,topic75,topic76,topic77,topic78,topic79,topic80,topic81,topic82,topic83,topic84,topic85,topic86,topic87,topic88,topic89,topic90,topic91,topic92,topic93,topic94,topic95,topic96,topic97,topic98,topic99,topic100,topic101,topic102,topic103,topic104,topic105,topic106,topic107,topic108,topic109,topic110,topic111,topic112,topic113,topic114,topic115,topic116,topic117,topic118,topic119
Eternal_Threads_review:_Eastenders_meets_time_travel_meets_electrical_safety,-0.066564,0.028993,0.015696,-0.068567,-0.004831,-0.040643,0.024112,0.013734,-0.113184,-0.039111,-0.000390,-0.050224,0.066446,-0.070505,-0.007473,-0.019492,-0.065916,-0.081967,-0.033730,0.016511,-0.020744,0.028383,0.028854,0.005221,-0.030879,0.019189,0.031577,-0.039801,-0.032103,0.014689,0.018947,0.006884,0.059610,-0.021486,-0.063805,-0.020856,0.036682,0.008526,-0.027129,0.021623,0.013774,-0.036324,-0.001161,-0.058783,0.055954,0.016229,-0.034921,0.028923,0.044509,-0.112991,0.047631,-0.045454,-0.022798,0.023655,-0.034261,0.008391,0.092429,0.093063,-0.021066,-0.027813,-0.038141,0.000967,-0.010683,-0.003420,0.000577,0.039364,-0.048107,0.023843,-0.009861,-0.035704,-0.026409,0.055116,-0.002855,-0.062330,-0.027776,-0.031482,-0.008915,0.002643,-0.018467,0.039767,-0.038769,-0.136715,0.001738,-0.025990,0.015657,-0.058311,0.044164,0.000770,-0.107852,0.020816,-0.031292,-0.059512,0.039478,-0.032755,0.082689,-0.082867,0.012748,0.040659,-0.002148,-0.065027,-0.063247,-0.106941,-0.014447,-0.037653,0.084618,0.024528,0.099964,0.083397,-0.089686,0.019369,0.098897,0.016730,-0.008205,0.026370,0.012599,-0.000952,-0.036569,0.074345,0.088231,-0.062229
ElecHead_review:_a_clever_puzzle_platformer_with_real_spark,-0.094091,0.067381,-0.000788,0.042958,0.029067,0.080380,-0.161202,0.130524,0.044597,-0.116874,0.101713,0.002516,0.042289,0.021354,0.027785,-0.015461,0.013931,-0.002029,-0.025119,-0.076514,-0.078127,0.050267,-0.054562,-0.000824,-0.031590,0.017721,0.035271,0.007513,0.007293,-0.008594,-0.025667,0.003147,0.065021,-0.025260,-0.011296,-0.069959,0.014903,0.031741,-0.045222,-0.002497,-0.017140,-0.019463,0.030925,-0.007623,0.021363,-0.015943,0.059424,0.019738,0.036723,0.011021,-0.012599,0.058743,-0.035677,0.022142,0.017045,0.001305,0.035044,0.059105,0.010893,0.013827,0.021540,-0.046877,0.035598,0.054895,0.007291,-0.005649,-0.021044,-0.034223,0.040772,-0.000760,0.011050,-0.018208,0.025243,0.003211,-0.024875,-0.020468,-0.054282,-0.040323,0.005330,0.013063,0.045484,-0.025021,0.020612,0.005264,0.006105,0.048454,0.020971,0.002845,0.051983,0.099977,-0.019895,-0.025502,0.037565,0.011602,-0.009664,0.022093,-0.035564,-0.000995,-0.035328,0.073723,0.047546,-0.027845,0.027398,0.047573,-0.026822,0.008688,0.008734,-0.009671,-0.050204,-0.035532,0.015078,0.066664,-0.016305,-0.031903,0.016887,-0.035634,0.071573,0.010554,-0.056217,-0.030705
Windjammers_2_review:_arcade_action_in_simple_and_enthralling_form,-0.033960,0.001919,-0.019701,0.018890,0.042373,-0.011837,0.045303,-0.003907,0.033922,-0.033089,-0.003239,-0.017369,-0.085977,-0.024816,-0.053516,-0.002110,0.013408,0.017446,-0.059295,-0.021626,0.022864,-0.000485,-0.023355,-0.079069,0.011449,-0.018413,-0.013104,-0.021608,0.023664,0.011765,0.023819,-0.004958,-0.028666,-0.007781,0.023711,0.004139,-0.013429,0.103827,-0.019490,0.004700,-0.064456,0.034157,0.012409,-0.003871,0.021941,-0.043834,0.016697,0.036533,0.000243,-0.020231,-0.005455,0.033926,0.016672,-0.000971,-0.025240,-0.010943,0.053030,0.0

In [21]:
svd_topic_vectors_df.sort_values(by=['topic0'], ascending=False)

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,topic11,topic12,topic13,topic14,topic15,topic16,topic17,topic18,topic19,topic20,topic21,topic22,topic23,topic24,topic25,topic26,topic27,topic28,topic29,topic30,topic31,topic32,topic33,topic34,topic35,topic36,topic37,topic38,topic39,topic40,topic41,topic42,topic43,topic44,topic45,topic46,topic47,topic48,topic49,topic50,topic51,topic52,topic53,topic54,topic55,topic56,topic57,topic58,topic59,topic60,topic61,topic62,topic63,topic64,topic65,topic66,topic67,topic68,topic69,topic70,topic71,topic72,topic73,topic74,topic75,topic76,topic77,topic78,topic79,topic80,topic81,topic82,topic83,topic84,topic85,topic86,topic87,topic88,topic89,topic90,topic91,topic92,topic93,topic94,topic95,topic96,topic97,topic98,topic99,topic100,topic101,topic102,topic103,topic104,topic105,topic106,topic107,topic108,topic109,topic110,topic111,topic112,topic113,topic114,topic115,topic116,topic117,topic118,topic119
Voice_Of_Cards:_The_Isle_Dragon_Roars_review:_a_JRPG_in_card_game_cosplay,0.350907,0.356042,-0.063792,0.014996,-0.024830,-0.078229,0.001898,-0.012003,0.020670,0.039791,0.088517,0.065338,-0.017196,0.043643,0.037936,0.002862,-0.060496,0.038720,0.051105,-0.004791,0.026544,-0.021356,0.013312,-0.000217,-0.030850,0.032903,-0.016958,0.013766,-0.023957,-0.000838,-0.029471,-0.029645,-0.000499,0.012313,-0.018019,0.026320,0.003472,-0.051582,-0.032819,-0.018461,0.041110,0.004694,-0.007508,-0.062531,-0.054427,-0.061268,-0.002807,0.003633,-0.060840,-0.033147,0.001343,0.038305,0.072429,-0.014741,-0.027521,0.002909,-0.004307,0.031528,0.006397,-0.029829,0.015816,0.060340,0.020917,0.042721,-0.049162,0.047575,0.020066,-0.004984,-0.015314,0.012759,-0.014135,0.040759,0.021680,0.024877,-0.002858,-0.012370,0.021806,-0.001491,0.031658,-0.004031,0.001549,0.008461,0.026076,-0.009775,0.035069,-0.030430,0.026971,-0.065956,-0.021757,0.047862,-0.036893,0.001007,-0.001105,-0.016571,-0.033593,-0.008044,-0.011271,0.041685,-0.034694,0.000771,0.011043,-0.027344,-0.010538,-0.026394,-0.024159,-0.084088,0.040524,0.010886,-0.029349,0.030697,0.017862,0.003454,-0.025618,-0.046247,0.027762,-0.015557,-0.011848,-0.019962,0.015640,-0.018992
ORX_review_early_access:_a_moreish_tower_defence_roguelike_that_already_has_strong_foundations,0.333112,0.242685,0.103982,0.023724,0.050035,0.000574,-0.044427,-0.067871,-0.029658,0.002387,0.025774,-0.020210,0.019726,0.020221,-0.048074,0.040194,-0.046022,0.034651,-0.040981,0.039442,-0.037860,0.023399,-0.026083,0.060988,0.013208,0.056502,0.012193,0.060953,-0.038393,-0.011643,-0.028622,-0.000055,0.018789,0.017366,-0.073124,0.011355,0.016669,0.003090,-0.031037,-0.034351,-0.024460,-0.021351,0.023374,0.064062,0.040710,-0.005258,-0.016991,-0.019580,0.054428,0.002824,0.011563,0.050071,-0.020946,0.062209,0.029795,-0.017675,0.031467,-0.043764,0.009769,-0.007398,-0.005631,0.009749,-0.001882,0.007151,-0.007917,-0.046416,-0.011835,0.048793,-0.045534,0.028852,0.026718,-0.069940,-0.010219,0.000762,0.061987,0.030021,0.011519,0.025889,-0.016705,0.010317,-0.004475,-0.036284,-0.012986,0.048190,0.036878,0.009939,-0.040208,0.047487,0.026569,-0.078961,-0.023397,-0.074399,0.024332,0.010195,-0.014477,0.053877,-0.014972,-0.029052,-0.024394,0.027299,0.030698,0.000377,0.001660,-0.032172,0.007080,-0.047186,-0.003272,0.006876,-0.001325,-0.000510,0.008858,-0.009819,0.079510,0.037836,-0.047266,0.071282,-0.015613,0.044222,0.055948,-0.021864
Foretales_review:_channelling_Hand_Of_Fate's_tabletop_deckbuilding_to_great_effect,0.324083,0.462813,0.066613,0.068480,0.035753,-0.145597,-0.048494,-0.009789,-0.006060,-0.000891,-0.014806,0.025048,0.015242,-0.003501,0.058270,0.024917,-0.041457,0.002172,0.056742,-0.023269,-0.058485,0.042469,0.023538,-0.007776,-0.012297,0.065913,0.006850,-0.013416,0.007408,0.006527,0.001905,-0.040160,0.034508,-0.012155,-0.020389,-0.005064,0.002161,-0.010849,-0.049952,0.004030,-0.007585,-0.007760,-0.001919,-0.001238,-0.044538,-0.034194,-0.044904,-0.010905,0.010965,-0.004030,0.025795,-0.018902,0.00

##### further exploration 

- evaluation of model & find the right num of topic / perplexity and coherence score 
  - reference 1: https://github.com/IkshitaMishra/TopicModelling-LSA-LDA/blob/master/Newspaper%20Topic%20Modelling.ipynb
  - reference 2: https://stackoverflow.com/questions/69730428/how-do-i-find-coherence-score-for-lsa-and-lda-for-sklearn-models
  - reference 3: https://www.baeldung.com/cs/topic-modeling-coherence-score
  - reference 4: https://finale.seas.harvard.edu/files/finale/files/assessing_topic_model_relevance.pdf
- visualisation of topic modelling
  - reference 5: https://stackoverflow.com/questions/51262950/visualization-for-output-of-topic-modelling
  - reference 6: https://github.com/x-tabdeveloping/topicwizard 

I haven't had enough time to explore the above content, it is not included in the essay, references are included for future exploration